In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime
import random 

In [2]:
clmns = [ '12 Open', '12 High', '12 Low', '12 Close', '12 Volume', '11 Open', '11 High', '11 Low', '11 Close', '11 Volume', '10 Open', '10 High', '10 Low', '10 Close', '10 Volume', '9 Open', '9 High', '9 Low', '9 Close', '9 Volume', '8 Open', '8 High', '8 Low', '8 Close', '8 Volume', '7 Open', '7 High', '7 Low', '7 Close', '7 Volume', '6 Open', '6 High', '6 Low', '6 Close', '6 Volume', '5 Open', '5 High', '5 Low', '5 Close', '5 Volume', '4 Open', '4 High', '4 Low', '4 Close', '4 Volume', '3 Open', '3 High', '3 Low', '3 Close', '3 Volume', '2 Open', '2 High', '2 Low', '2 Close', '2 Volume', '1 Open', '1 High', '1 Low', '1 Close', '1 Volume',"suggestion"]
clmnsws = ['12 Open', '12 High', '12 Low', '12 Close', '12 Volume', '11 Open', '11 High', '11 Low', '11 Close', '11 Volume', '10 Open', '10 High', '10 Low', '10 Close', '10 Volume', '9 Open', '9 High', '9 Low', '9 Close', '9 Volume', '8 Open', '8 High', '8 Low', '8 Close', '8 Volume', '7 Open', '7 High', '7 Low', '7 Close', '7 Volume', '6 Open', '6 High', '6 Low', '6 Close', '6 Volume', '5 Open', '5 High', '5 Low', '5 Close', '5 Volume', '4 Open', '4 High', '4 Low', '4 Close', '4 Volume', '3 Open', '3 High', '3 Low', '3 Close', '3 Volume', '2 Open', '2 High', '2 Low', '2 Close', '2 Volume', '1 Open', '1 High', '1 Low', '1 Close', '1 Volume']
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
   'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(13,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-12][0],
                df[each-12][1],
                df[each-12][2],
                df[each-12][3],
                df[each-12][4],
                df[each-11][0],
                df[each-11][1],
                df[each-11][2],
                df[each-11][3],
                df[each-11][4],
                df[each-10][0],
                df[each-10][1],
                df[each-10][2],
                df[each-10][3],
                df[each-10][4],
                df[each-9][0],
                df[each-9][1],
                df[each-9][2],
                df[each-9][3],
                df[each-9][4],
                df[each-8][0],
                df[each-8][1],
                df[each-8][2],
                df[each-8][3],
                df[each-8][4],
                df[each-7][0],
                df[each-7][1],
                df[each-7][2],
                df[each-7][3],
                df[each-7][4],
                df[each-6][0],
                df[each-6][1],
                df[each-6][2],
                df[each-6][3],
                df[each-6][4],
                df[each-5][0],
                df[each-5][1],
                df[each-5][2],
                df[each-5][3],
                df[each-5][4],
                df[each-4][0],
                df[each-4][1],
                df[each-4][2],
                df[each-4][3],
                df[each-4][4],
                df[each-3][0],
                df[each-3][1],
                df[each-3][2],
                df[each-3][3],
                df[each-3][4],
                df[each-2][0],
                df[each-2][1],
                df[each-2][2],
                df[each-2][3],
                df[each-2][4],
                df[each-1][0],
                df[each-1][2],
                df[each-1][1],
                df[each-1][3],
                df[each-1][4],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(-6,6))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object,columns = clmnsws)
  scaled["suggestion"] = last_column
  return scaled
def column_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  df_scaled = scaler.fit_transform(df.to_numpy())
  df_scaled = pd.DataFrame(df_scaled,columns = clmnsws)
  df_scaled["suggestion"] = last_column
  return df_scaled
def process(df): 
      df = df.dropna()
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df,columns = clmns)
      df = row_scaler(df)
      #df = column_scaler(df)
      return df

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
              pass
           else:
               if data.shape[0] > 14:
                   unattached_dfs.append(process(data))
    symbols = []
    symbol = []
    data = []

    data = pd.concat(unattached_dfs)
    data = data.astype(float)

    #xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,1000)}.csv")
    data = []
   # return xTrain, xTest, yTrain, yTest

#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
symbols = read_syms_from_txt()
#symbols = get_crypto_syms()

pieces = 10
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays:
   get_data(symbols)
    
#Train, xTest, yTrain, yTest= get_data(symbols)

In [3]:
data = pd.read_parquet("/content/drive/MyDrive/loll1.parquet")
data.drop("Unnamed: 0",axis=1,inplace=True)
xTrain, xTest, yTrain, yTest= spliting(data)
data

(4065579, 60) (4065579,)
(451731, 60) (451731,)


,12 Open,12 High,12 Low,12 Close,12 Volume,11 Open,11 High,11 Low,11 Close,11 Volume,...,2 High,2 Low,2 Close,2 Volume,1 Open,1 High,1 Low,1 Close,1 Volume,suggestion
0,1.389476,2.842105,-5.052634,2.336842,2.336842,1.831579,3.473686,-6.000000,2.526314,2.526314,...,5.684209,0.694735,3.978949,3.978949,3.789471,1.452629,6.000000,4.105262,4.105262,0.0
1,1.831579,3.473686,-6.000000,2.526314,2.526314,2.715786,4.092631,-2.778951,3.410527,3.410527,...,6.000000,1.452629,4.105262,4.105262,2.842105,-1.263157,4.389473,1.010526,1.010526,1.0
2,1.510789,3.392806,-6.000000,2.460434,2.460434,2.374101,4.273383,-4.014385,-0.474819,-0.474819,...,3.798561,-3.928052,-0.820142,-0.820142,1.683454,-1.338131,3.410071,3.410071,3.410071,1.0
3,2.123077,4.153848,-4.707689,-0.923076,-0.923076,-0.830766,1.384616,-5.723079,0.184615,0.184615,...,3.230768,-1.846156,3.230768,3.230768,3.230768,-3.046148,5.344615,3.876928,3.876928,1.0
4,-2.726740,-1.323916,-5.824648,-2.083780,-2.083780,-2.726740,-0.996592,-6.000000,-2.492936,-2.492936,...,1.183633,-4.129563,0.254264,0.254264,-1.557720,-3.252800,6.000000,3.176816,3.176816,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191987,4.057190,6.000000,3.767219,5.415224,5.415224,5.415224,5.420058,2.559003,3.472414,3.472414,...,-3.578736,-5.231575,-4.622635,-4.622635,-4.806282,-6.000000,-4.748288,-5.908176,-5.908176,0.0
1191988,5.995366,6.000000,3.257144,4.132819,4.132819,4.248649,4.753667,2.900385,4.160617,4.160617,...,-3.748262,-4.948262,-4.860232,-4.860232,-5.397684,-6.000000,-5.045559,-5.944402,-5.944402,0.0
1191989,4.587955,5.109693,3.195054,4.497009,4.497009,5.617074,6.000000,4.650180,5.731952,5.731952,...,-5.013960,-6.000000,-5.942561,-5.942561,-5.080972,-5.985641,-4.248105,-5.803750,-5.803750,1.0
1191990,5.617074,6.000000,4.650180,5.731952,5.731952,5.703230,5.708017,3.913043,3.960908,3.960908,...,-4.248105,-5.985641,-5.803750,-5.803750,-5.597926,-5.708017,-4.631033,-5.171919,-5.171919,0.0


In [5]:
data = []

In [16]:
model = Sequential()
adam =tf.keras.optimizers.Adam(learning_rate=0.01)


model.add(Dense(3000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(3096, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_55 (Dense)            (None, 3000)              183000    
                                                                 
 dense_56 (Dense)            (None, 3096)              9291096   
                                                                 
 dense_57 (Dense)            (None, 2048)              6342656   
                                                                 
 dense_58 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_59 (Dense)            (None, 512)               524800    
                                                                 
 dense_60 (Dense)            (None, 1)                 513       
                                                                 
Total params: 18,440,241
Trainable params: 18,440,241


In [17]:
model.fit(xTrain,yTrain,epochs=30,batch_size=50000,validation_data=(xTest,yTest))

Epoch 1/30
82/82 [==============================] - 207s 3s/step - loss: 237.2658 - accuracy: 0.5593 - val_loss: 0.6626 - val_accuracy: 0.5927
Epoch 2/30
82/82 [==============================] - 206s 3s/step - loss: 0.6605 - accuracy: 0.5903 - val_loss: 0.6556 - val_accuracy: 0.5926
Epoch 3/30
82/82 [==============================] - 204s 2s/step - loss: 0.6566 - accuracy: 0.5899 - val_loss: 0.6551 - val_accuracy: 0.5927
Epoch 4/30
82/82 [==============================] - 204s 2s/step - loss: 0.6558 - accuracy: 0.5894 - val_loss: 0.6553 - val_accuracy: 0.5906
Epoch 5/30
82/82 [==============================] - 206s 3s/step - loss: 0.6565 - accuracy: 0.5873 - val_loss: 0.6578 - val_accuracy: 0.5922
Epoch 6/30
82/82 [==============================] - 204s 2s/step - loss: 0.6556 - accuracy: 0.5890 - val_loss: 0.6549 - val_accuracy: 0.5922
Epoch 7/30
59/82 [====================>.........] - ETA: 55s - loss: 0.6534 - accuracy: 0.5897

KeyboardInterrupt: ignored

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
